In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
def create_modified_filter(data_quality_filter):
    # Extract the tag IDs from the excluded list
    excluded_tag_ids = [tag['id'] for tag in data_quality_filter['object_filter']['tags']['value']['excluded']]

    # Create the modified filter
    modified_filter = {
        'tags': {
            'modifier': 'INCLUDES_ALL',
            'value': [],
            'depth': -1,
            'excludes': excluded_tag_ids,
        }
    }
    return modified_filter

def get_scene_count(filter):
    result = stash.call_GQL("""query FindScenes($scene_filter: SceneFilterType) {
        findScenes(scene_filter: $scene_filter) {
            count
        }
    }""", variables={"scene_filter": filter})
    return result['findScenes']['count']

In [ ]:
saved_filters = stash.call_GQL("""query FindSavedFilters {
    findSavedFilters {
        id
        mode
        name
        filter
        object_filter
        ui_options
    }
}""")

In [ ]:
data_quality_filters = [filter for filter in saved_filters['findSavedFilters'] if 'data quality' in filter['name'].lower()]
data_quality_filters

In [ ]:
# Loop through all data quality filters and print out name and count of matching scenes
for filter in data_quality_filters:
    name = filter['name']
    modified_filter = create_modified_filter(filter)
    scene_count = get_scene_count(modified_filter)
    print(f"{name}: {scene_count} scenes")
